In [16]:
import pandas as pd

def map_property_type(area):
    if area < 900:
        return 'Condo/Co-op'
    elif area < 1500:
        return 'Townhouse'
    elif area < 2500:
        return 'Single Family Residential'
    else:
        return 'All Residential'  # fallback for large or unknown

#philly['property_type'] = philly['total_area'].apply(map_property_type)

In [20]:
philly = pd.read_csv("philly_property_data.csv")
redfin = pd.read_csv("redfin_with_crime.csv")

philly['property_type'] = philly['total_area'].apply(map_property_type)

# Aggregate redfin by zip_code + property_type
redfin_agg = redfin.groupby(['zip_code', 'property_type']).mean().reset_index()

# Merge redfin into philly on both zip_code and property_type
merged = philly.merge(redfin_agg, on=['zip_code', 'property_type'], how='left')



is_numeric = redfin.select_dtypes(include='number')
numeric_cols = is_numeric.columns.drop('zip_code')
redfin_zip_only = redfin.groupby('zip_code')[numeric_cols].mean().reset_index()


redfin_zip_only_renamed = redfin_zip_only.rename(columns={
    col: col + '_ziponly' for col in redfin_zip_only.columns if col != 'zip_code'
})

philly_fallback = philly.merge(redfin_zip_only_renamed, on='zip_code', how='left')

# Fill missing values from fallback
cols_to_fill = [
    'avg_price_sold', 'percent_above_list', 'avg_price_per_sqft', 'new_listings',
    'pending_sales', 'homes_sold', 'sale_to_list_ratio', 'active_inventory',
    'avg_days_on_market', 'crimes_per_100k',
    'avg_listed_price', 'avg_list_price_per_sqft', 'off_market_2w'
]

for col in cols_to_fill:
    fallback_col = col + '_ziponly'
    if fallback_col in philly_fallback.columns:
        merged[col] = merged[col].fillna(philly_fallback[fallback_col])

In [21]:
print(merged.isnull().sum().sort_values(ascending=False).head(10))

avg_price_sold         0
avg_price_per_sqft     0
year_built_estimate    0
year_built             0
total_area             0
total_livable_area     0
number_stories         0
number_of_bathrooms    0
number_of_bedrooms     0
garage_spaces          0
dtype: int64


In [22]:
missing_rows = merged[merged['avg_price_sold'].isna()][['zip_code', 'property_type']]

# View unique problematic pairs
print(missing_rows.drop_duplicates())

Empty DataFrame
Columns: [zip_code, property_type]
Index: []


In [23]:
print(redfin.groupby(['zip_code', 'property_type']).size().reset_index().head())

   zip_code              property_type    0
0       501            All Residential   27
1       501  Single Family Residential   27
2      1001            All Residential  157
3      1001                Condo/Co-op  157
4      1001    Multi-Family (2-4 Unit)  137


In [24]:
merged['sale_date'] = pd.to_datetime(merged['sale_date'])
merged['sale_day'] = merged['sale_date'].dt.day
merged['sale_month'] = merged['sale_date'].dt.month
merged['sale_year'] = merged['sale_date'].dt.year

merged['est_listed_date'] = merged['sale_date'] - pd.to_timedelta(merged['avg_days_on_market'], unit='D')

merged['est_list_day'] = merged['est_listed_date'].dt.day
merged['est_list_month'] = merged['est_listed_date'].dt.month
merged['est_list_year'] = merged['est_listed_date'].dt.year

# Easton TOM in Days
merged['Est TOM in Days'] = (merged['sale_date'] - merged['est_listed_date']).dt.days

In [25]:
final = merged[['zip_code', 'central_air', 'garage_spaces', 
    'number_of_bedrooms', 'number_of_bathrooms', 
    'number_stories', 'total_livable_area', 'total_area', 
    'year_built', 'has_basements', 'sale_price', 
    'active_inventory', 'new_listings', 'pending_sales', 
    'homes_sold', 'sale_to_list_ratio', 
    'percent_above_list', 'avg_days_on_market', 'crimes_per_100k', 'off_market_2w', 
    'avg_list_price_per_sqft','avg_listed_price', 
    'avg_price_per_sqft', 'avg_price_sold', 'Est TOM in Days'
]]

In [26]:
#final = merged.drop(columns = ['sale_date','property_type','est_listed_date'])

final.to_csv("philly_final_dataset_Reduced.csv", index=False)

In [10]:
merged.head()

,zip_code,sale_date,view_scale,heater_scale,exterior_condition,interior_condition,central_air,basement_scale,garage_spaces,number_of_bedrooms,...,avg_listed_price,avg_price_per_sqft,avg_price_sold,sale_day,sale_month,sale_year,est_listed_date,est_list_day,est_list_month,est_list_year
0,19142,2025-03-09,7,1.0,4,4,False,1.0,0,7,...,84758.977564,65.224775,74014.641026,9,3,2025,2025-01-16 02:23:04.615353600,16,1,2025
1,19104,2025-01-08,7,1.0,4,4,False,1.0,0,4,...,332588.855469,138.411437,303111.900735,8,1,2025,2024-09-03 22:34:01.791004800,3,9,2024
2,19138,2024-12-20,7,4.0,4,4,False,1.0,0,3,...,127099.621795,97.912553,125237.701923,20,12,2024,2024-10-28 01:09:13.846147200,28,10,2024
3,19125,2025-01-17,0,7.0,1,1,True,0.0,1,3,...,348528.166667,285.347580,331619.483974,17,1,2025,2024-10-26 00:18:42.077923200,26,10,2024
4,19145,2020-01-03,7,1.0,4,4,False,1.0,0,3,...,344790.550000,224.781795,353435.219697,3,1,2020,2019-08-10 21:27:16.363641600,10,8,2019
